# EE 641, Spring 2023 - Homework #1
## Region-based convolutional neural networks (R-CNN)

### Overview

Fine-tune a PyTorch R-CNN model to extract text from natural scenes (see figure below).  Given a scene input your model should output a list of bounding boxes corresponding to each text region.  

Prepare:
  1. validation figures that show machine-predicted bounding boxes vs. the ground-truth boxes
  1. loss-curves that show component-wise R-CNN losses as a function of epoch

![Text regions](attachment:4_images_with_boxes_and_text-2.png)

### Dataset

Train your model using the **Text** MS-COCO dataset.  Text MS-COCO is a subset of the original MS-COCO dataset (2014) with additional text annotations.  Read more: [COCO Text](https://bgshih.github.io/cocotext/)

We prepared a copy of the dataset with some minimal cleaning: https://data.usc-ece.com. Download the dataset and extract (unzip) the images.  If you are using CoLab you _can_ download each session using a direct url but it is better to copy the file to your GDrive and mount into your notebook:

```
from google.colab import drive
drive.mount('/content/drive')
# > Mounted at /content/drive

!unzip /content/drive/MyDrive/ee641/datasets/data.zip

# (optional, cleanup)
!rm data.zip
!rm -rf /content/sample_data
```

Then download the JSON annotation file that includes target bounding boxes and contained content.

```
Downloading...
From: https://data.usc-ece.com/EE641/HW1/COCO_Text.json
To: /content/COCO_Text.json`
100% 67.4M/67.4M [00:01<00:00, 56.6MB/s]
```

In [1]:
!wget https://data.usc-ece.com/EE641/HW1/mscoco_text_cleaned_v01.zip

--2023-02-27 08:11:43--  https://data.usc-ece.com/EE641/HW1/mscoco_text_cleaned_v01.zip
Resolving data.usc-ece.com (data.usc-ece.com)... 18.172.170.20, 18.172.170.80, 18.172.170.21, ...
Connecting to data.usc-ece.com (data.usc-ece.com)|18.172.170.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1465384901 (1.4G) [application/zip]
Saving to: ‘mscoco_text_cleaned_v01.zip’

_text_cleaned_v01.z  11%[=>                  ] 159.87M  43.0MB/s    eta 29s    ^C


In [9]:
!wget https://data.usc-ece.com/EE641/HW1/COCO_Text.json

--2023-02-28 08:12:36--  https://data.usc-ece.com/EE641/HW1/COCO_Text.json
Resolving data.usc-ece.com (data.usc-ece.com)... 18.161.97.48, 18.161.97.52, 18.161.97.121, ...
Connecting to data.usc-ece.com (data.usc-ece.com)|18.161.97.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67396854 (64M) [application/json]
Saving to: ‘COCO_Text.json’

COCO_Text.json      100%[===================>]  64.27M  14.5MB/s    in 5.5s    

2023-02-28 08:12:43 (11.6 MB/s) - ‘COCO_Text.json’ saved [67396854/67396854]



In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!unzip /content/drive/MyDrive/ee641/datasets/data.zip

Streaming output truncated to the last 5000 lines.
  inflating: data/train/COCO_train2014_000000435897.jpg  
  inflating: data/train/COCO_train2014_000000436161.jpg  
  inflating: data/train/COCO_train2014_000000437605.jpg  
  inflating: data/train/COCO_train2014_000000437849.jpg  
  inflating: data/train/COCO_train2014_000000439268.jpg  
  inflating: data/train/COCO_train2014_000000440792.jpg  
  inflating: data/train/COCO_train2014_000000441850.jpg  
  inflating: data/train/COCO_train2014_000000442911.jpg  
  inflating: data/train/COCO_train2014_000000443209.jpg  
  inflating: data/train/COCO_train2014_000000443390.jpg  
  inflating: data/train/COCO_train2014_000000443941.jpg  
  inflating: data/train/COCO_train2014_000000444453.jpg  
  inflating: data/train/COCO_train2014_000000444591.jpg  
  inflating: data/train/COCO_train2014_000000445143.jpg  
  inflating: data/train/COCO_train2014_000000445325.jpg  
  inflating: data/train/COCO_train2014_000000445338.jpg  
  inflating: data/tra

## Helpers (Complete, but you may modify)

In [11]:
#@title Imports
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch
import os
import cv2
import numpy as np
from PIL import Image
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator, RPNHead, RegionProposalNetwork
import torch
import shutil
import time

In [12]:
#@title COCO_Text class: Handles the annotation file
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

__author__ = 'andreasveit'
__version__ = '1.1'
# Interface for accessing the COCO-Text dataset.

# COCO-Text is a large dataset designed for text detection and recognition.
# This is a Python API that assists in loading, parsing and visualizing the 
# annotations. The format of the COCO-Text annotations is also described on 
# the project website http://vision.cornell.edu/se3/coco-text/. In addition to this API, please download both
# the COCO images and annotations.
# This dataset is based on Microsoft COCO. Please visit http://mscoco.org/
# for more information on COCO, including for the image data, object annotatins
# and caption annotations. 

# An alternative to using the API is to load the annotations directly
# into Python dictionary:
# with open(annotation_filename) as json_file:
#     coco_text = json.load(json_file)
# Using the API provides additional utility functions.

# The following API functions are defined:
#  COCO_Text  - COCO-Text api class that loads COCO annotations and prepare data structures.
#  getAnnIds  - Get ann ids that satisfy given filter conditions.
#  getImgIds  - Get img ids that satisfy given filter conditions.
#  loadAnns   - Load anns with the specified ids.
#  loadImgs   - Load imgs with the specified ids.
#  showAnns   - Display the specified annotations.
#  loadRes    - Load algorithm results and create API for accessing them.
# Throughout the API "ann"=annotation, "cat"=category, and "img"=image.

# COCO-Text Toolbox.        Version 1.1
# Data and  paper available at:  http://vision.cornell.edu/se3/coco-text/
# Code based on Microsoft COCO Toolbox Version 1.0 by Piotr Dollar and Tsung-Yi Lin
# extended and adapted by Andreas Veit, 2016.
# Licensed under the Simplified BSD License [see bsd.txt]

import json
import datetime
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle, PathPatch
from matplotlib.path import Path
import numpy as np
import copy
import os

class COCO_Text:
    def __init__(self, annotation_file=None):
        """
        Constructor of COCO-Text helper class for reading and visualizing annotations.
        :param annotation_file (str): location of annotation file
        :return:
        """
        # load dataset
        self.dataset = {}
        self.anns = {}
        self.imgToAnns = {}
        self.catToImgs = {}
        self.imgs = {}
        self.cats = {}
        self.val = []
        self.test = []
        self.train = []
        if not annotation_file == None:
            assert os.path.isfile(annotation_file), "file does not exist"
            print('loading annotations into memory...')
            time_t = datetime.datetime.utcnow()
            dataset = json.load(open(annotation_file, 'r'))
            print(datetime.datetime.utcnow() - time_t)
            self.dataset = dataset
            self.createIndex()

    def createIndex(self):
        # create index
        print('creating index...')
        self.imgToAnns = {int(cocoid): self.dataset['imgToAnns'][cocoid] for cocoid in self.dataset['imgToAnns']}
        self.imgs      = {int(cocoid): self.dataset['imgs'][cocoid] for cocoid in self.dataset['imgs']}
        self.anns      = {int(annid): self.dataset['anns'][annid] for annid in self.dataset['anns']}
        self.cats      = self.dataset['cats']
        self.val       = [int(cocoid) for cocoid in self.dataset['imgs'] if self.dataset['imgs'][cocoid]['set'] == 'val']
        self.test      = [int(cocoid) for cocoid in self.dataset['imgs'] if self.dataset['imgs'][cocoid]['set'] == 'test']
        self.train     = [int(cocoid) for cocoid in self.dataset['imgs'] if self.dataset['imgs'][cocoid]['set'] == 'train']
        print('index created!')

    def info(self):
        """
        Print information about the annotation file.
        :return:
        """
        for key, value in self.dataset['info'].items():
            print('%s: %s'%(key, value))

    def filtering(self, filterDict, criteria):
        return [key for key in filterDict if all(criterion(filterDict[key]) for criterion in criteria)]

    def getAnnByCat(self, properties):
        """
        Get ann ids that satisfy given properties
        :param properties (list of tuples of the form [(category type, category)] e.g., [('readability','readable')] 
            : get anns for given categories - anns have to satisfy all given property tuples
        :return: ids (int array)       : integer array of ann ids
        """
        return self.filtering(self.anns, [lambda d, x=a, y=b:d[x] == y for (a,b) in properties])

    def getAnnIds(self, imgIds=[], catIds=[], areaRng=[]):
        """
        Get ann ids that satisfy given filter conditions. default skips that filter
        :param imgIds  (int array)     : get anns for given imgs
               catIds  (list of tuples of the form [(category type, category)] e.g., [('readability','readable')] 
                : get anns for given cats
               areaRng (float array)   : get anns for given area range (e.g. [0 inf])
        :return: ids (int array)       : integer array of ann ids
        """
        imgIds = imgIds if type(imgIds) == list else [imgIds]
        catIds = catIds if type(catIds) == list else [catIds]

        if len(imgIds) == len(catIds) == len(areaRng) == 0:
            anns = list(self.anns.keys())
        else:
            if not len(imgIds) == 0:
                anns = sum([self.imgToAnns[imgId] for imgId in imgIds if imgId in self.imgToAnns],[])
            else:
                anns = list(self.anns.keys())
            anns = anns if len(catIds)  == 0 else list(set(anns).intersection(set(self.getAnnByCat(catIds)))) 
            anns = anns if len(areaRng) == 0 else [ann for ann in anns if self.anns[ann]['area'] > areaRng[0] and self.anns[ann]['area'] < areaRng[1]]
        return anns

    def getImgIds(self, imgIds=[], catIds=[]):
        '''
        Get img ids that satisfy given filter conditions.
        :param imgIds (int array) : get imgs for given ids
        :param catIds (int array) : get imgs with all given cats
        :return: ids (int array)  : integer array of img ids
        '''
        imgIds = imgIds if type(imgIds) == list else [imgIds]
        catIds = catIds if type(catIds) == list else [catIds]

        if len(imgIds) == len(catIds) == 0:
            ids = list(self.imgs.keys())
        else:
            ids = set(imgIds)
            if not len(catIds) == 0:
                ids  = ids.intersection(set([self.anns[annid]['image_id'] for annid in self.getAnnByCat(catIds)]))
        return list(ids)

    def loadAnns(self, ids=[]):
        """
        Load anns with the specified ids.
        :param ids (int array)       : integer ids specifying anns
        :return: anns (object array) : loaded ann objects
        """
        if type(ids) == list:
            return [self.anns[id] for id in ids]
        elif type(ids) == int:
            return [self.anns[ids]]

    def loadImgs(self, ids=[]):
        """
        Load anns with the specified ids.
        :param ids (int array)       : integer ids specifying img
        :return: imgs (object array) : loaded img objects
        """
        if type(ids) == list:
            return [self.imgs[id] for id in ids]
        elif type(ids) == int:
            return [self.imgs[ids]]

    def showAnns(self, anns, show_polygon=False):
        """
        Display the specified annotations.
        :param anns (array of object): annotations to display
        :return: None
        """
        if len(anns) == 0:
            return 0
        ax = plt.gca()
        boxes = []
        color = []
        for ann in anns:
            c = np.random.random((1, 3)).tolist()[0]
            if show_polygon:
                tl_x, tl_y, tr_x, tr_y, br_x, br_y, bl_x, bl_y = ann['polygon']
                verts = [(tl_x, tl_y), (tr_x, tr_y), (br_x, br_y), (bl_x, bl_y), (0, 0)]
                codes = [Path.MOVETO, Path.LINETO, Path.LINETO, Path.LINETO, Path.CLOSEPOLY]
                path = Path(verts, codes)
                patch = PathPatch(path, facecolor='none')
                boxes.append(patch)
                left, top = tl_x, tl_y
            else:
                left, top, width, height = ann['bbox']
                boxes.append(Rectangle([left,top],width,height,alpha=0.4))
            color.append(c)
            if 'utf8_string' in list(ann.keys()):
                ax.annotate(ann['utf8_string'],(left,top-4),color=c)
        p = PatchCollection(boxes, facecolors=color, edgecolors=(0,0,0,1), linewidths=3, alpha=0.4)
        ax.add_collection(p)

    def loadRes(self, resFile):
        """
        Load result file and return a result api object.
        :param   resFile (str)     : file name of result file
        :return: res (obj)         : result api object
        """
        res = COCO_Text()
        res.dataset['imgs'] = [img for img in self.dataset['imgs']]

        print('Loading and preparing results...     ')
        time_t = datetime.datetime.utcnow()
        if type(resFile) == str:
            anns = json.load(open(resFile))
        else:
            anns = resFile
        assert type(anns) == list, 'results in not an array of objects'
        annsImgIds = [int(ann['image_id']) for ann in anns]
        
        if set(annsImgIds) != (set(annsImgIds) & set(self.getImgIds())):
            print('Results do not correspond to current coco set')
            print('skipping ', str(len(set(annsImgIds)) - len(set(annsImgIds) & set(self.getImgIds()))), ' images')
        annsImgIds = list(set(annsImgIds) & set(self.getImgIds()))

        res.imgToAnns = {cocoid : [] for cocoid in annsImgIds}
        res.imgs = {cocoid: self.imgs[cocoid] for cocoid in annsImgIds} 

        assert anns[0]['bbox'] != [], 'results have incorrect format'
        for id, ann in enumerate(anns):
            if ann['image_id'] not in annsImgIds:
                continue
            bb = ann['bbox']
            ann['area'] = bb[2]*bb[3]
            ann['id'] = id
            res.anns[id] = ann
            res.imgToAnns[ann['image_id']].append(id)
        print('DONE (t=%0.2fs)'%((datetime.datetime.utcnow() - time_t).total_seconds()))

        return res

### CocoDataset class (TODO)

TODO: `CocoDataset::__getitem__()`

In [13]:
#@title Dataset class: use images and annotation file to implement a dataset that can used with Dataloader

class CocoDataset(Dataset):
    def __init__(self, root_dir, annFile, transform=None, cuda=True):
        self.root_dir = root_dir
        self.transform = transform
        self.imgs = os.listdir(root_dir)
        # annotations
        self.ct = COCO_Text(annFile)
        self.imgIds = self.ct.getImgIds(imgIds=self.ct.train, 
                    catIds=[('legibility','legible'),('class','machine printed')])
        
        for imgId in self.imgIds:
            file_name = self.ct.loadImgs(imgId)[0]['file_name']
            if file_name not in self.imgs:
                self.imgIds.remove(imgId)
        # manual exclude
        self.imgIds.remove(275939)
        self.imgIds.remove(443671)

        # remaining images
        print(f"remaining images in ann file: {len(self.imgIds)}, remaining images in folder: {len(self.imgs)}")

        self.imgIds.sort()
        # sort the images in same order as the annotations
        self.imgs = [self.ct.loadImgs(imgId)[0]['file_name'] for imgId in self.imgIds]

        self.img_h = 224
        self.img_w = 224
        self.cuda = cuda

        
    def __len__(self):
        return len(self.imgs)
    
    def __getitem__(self, idx):
        # TODO: return image / bounding-box [list] pair for self.imgIds[idx]
        # resize the image to conform to your model input and transformed
        # bounding box coordinates
        
        # see COCO_Text helper (i.e., self.ct)
        # return image, target
        
        # raise NotImplementedError("CocoDataset::__getitem__()")

        # Solution
        img_name = os.path.join(self.root_dir, self.imgs[idx])
        image = Image.open(img_name)
        # get original image size
        w, h = image.size
        # square pad the image to img_h, img_w
        image = image.resize((self.img_w, self.img_h))

        if image.mode != 'RGB':
           image = image.convert('RGB')  

        if self.transform:
            image = self.transform(image)
        target = {}
        img = self.ct.loadImgs(self.imgIds[idx])[0]
        annIds = self.ct.getAnnIds(imgIds=img['id'])
        anns = self.ct.loadAnns(annIds)
        # scale the bounding boxes from original image size to img_h, img_w
        for ann in anns:
            ann['bbox'][0] = ann['bbox'][0] * self.img_w / w
            ann['bbox'][1] = ann['bbox'][1] * self.img_h / h
            ann['bbox'][2] = ann['bbox'][2] * self.img_w / w
            ann['bbox'][3] = ann['bbox'][3] * self.img_h / h

            assert ann['bbox'][2] > 0 and ann['bbox'][3] > 0

            # change 2 and 3
            ann['bbox'][2] = ann['bbox'][2] + ann['bbox'][0]
            ann['bbox'][3] = ann['bbox'][3] + ann['bbox'][1]
        target['boxes'] = torch.as_tensor([ann['bbox'] for ann in anns], dtype=torch.float32)
        target['labels'] = torch.ones((len(anns),), dtype=torch.int64)

        return image, target

        

# coalate_fn is used to collate the data into batches
def collate_fn(batch):
    images = []
    targets = []
    for item in batch:
        images.append(item[0])
        targets.append(item[1])
    images = torch.stack(images, 0)
    return images, targets

# Evaluate / region visualization (TODO)

Load four images and predict regions using model.
Create a figure that shows the images as subplots. 
Overlay model predicted regions (blue) and target regions (red).
Save the output as an image -- use epoch in filename to preserve order.

In [14]:
def evaluate(model, dataloader, device, epoch):
    # TODO: create four-panel subplots showing bounding boxes
    # eval on 4 images
    images, targets = next(iter(dataloader))
    images = list(image.to(device) for image in images)
    targets = [{epoch: v for epoch, v in t.items()} for t in targets]
    # number of boxes in each image
    num_boxes = [len(target['boxes']) for target in targets]

    # pred
    model.eval()
    pred = model(images)

    # plot the images and the bounding boxes
    plt.clf()
    for i in range(4):
        # clear the plot
        ax = plt.subplot(2, 2, i+1)
        img = images[i].permute(1, 2, 0)
        img = img.cpu().numpy()
        img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
        img = np.clip(img, 0, 1)
        plt.imshow(img)
        
        j = 0
        # select only num_boxes[i] boxes
        while j < num_boxes[i] and j < len(pred[i]['boxes']):
            box = pred[i]['boxes'][j]
            # convert to numpy
            box = box.detach().cpu().numpy()
            
            # if pred[i]['scores'][j] > 0.7:
            box_ = box
            rect = plt.Rectangle((box_[0] , box_[1]), box_[2] - box_[0], box_[3] - box_[1], fill=False, edgecolor='r', linewidth=1, alpha=pred[i]['scores'][j].item())
            ax.add_patch(rect)
            j += 1
        plt.axis('off')

        # actual bounding boxes
        for box in targets[i]['boxes']:
            box_ = box.detach().numpy()
            # rect = plt.Rectangle((box_[0], box_[1]), box_[2], box_[3], linewidth=1, edgecolor='b', facecolor='none')
            rect = plt.Rectangle((box_[0], box_[1]), box_[2] - box_[0], box_[3] - box_[1], fill=False, edgecolor='b', linewidth=1)

            ax.add_patch(rect)
        plt.axis('off')
    # save the image
    os.makedirs("images", exist_ok=True)
    plt.savefig(f"images/test_{epoch}.png")
    #raise NotImplementedError("evaluate()")


# Model factory (TODO)

Use a pretrained torchvision model. Keep training resources in mind when selecting a model. Make changes necessary to adapt the pretrained network to this dataset.  [Example](https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html)

In [15]:
def get_model():
    # TODO: create and return a PyTorch model
    fasterRCNN = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True)
    in_features = fasterRCNN.roi_heads.box_predictor.cls_score.in_features #get number of features
    fasterRCNN.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes = 2)
    fasterRCNN.roi_heads.fg_bg_sampler.batch_size_per_image = 24
    fasterRCNN.roi_heads.fg_bg_sampler.positive_fraction = 0.5
    return fasterRCNN
    #raise NotImplementedError("get_model()")

# Setup (TODO)

In [16]:
transform = transforms.Compose([
  # TODO: e.g. transforms.Normalize()
    # transforms.Resize((224, 224)),
    # SquarePad(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])
])

# TODO: dataset = CocoDataset(..., transform=transform)
dataset = CocoDataset(root_dir='/content/data/train', annFile='/content/COCO_Text.json', transform=transform)
# TODO: dataloader = DataLoader(..., collate_fn=collate_fn)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, num_workers=8, collate_fn=collate_fn)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = get_model()
# revise as needed
#device = torch.device('cpu')  Commented to use GPU 
model.to(device)

loading annotations into memory...
0:00:02.003325
creating index...
index created!
remaining images in ann file: 8738, remaining images in folder: 8738


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_MobileNet_V3_Large_FPN_Weights.COCO_

  0%|          | 0.00/74.2M [00:00<?, ?B/s]

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): FrozenBatchNorm2d(16, eps=1e-05)
        (2): Hardswish()
      )
      (1): InvertedResidual(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
            (1): FrozenBatchNorm2d(16, eps=1e-05)
            (2): ReLU(inplace=True)
          )
          (1): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): FrozenBatchNorm2d(16, eps=1e-05)
          )
        )
      )
      (2): InvertedResidual(
        (block):

# Training (TODO)

In [ ]:
# TODO: complete optimizer (optional, learning rate scheduler)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.00005, betas=(0.9, 0.999), weight_decay=0.0005)

num_epochs = 100

metric_collector1 = []
metric_collector2 = []
metric_collector3 = []
metric_collector4 = []

for epoch in range(num_epochs):
    for idx, data in enumerate(dataloader):
        model.train()
        # TODO: implement training loop: inference, backpropagate, update
        images = data[0].to(device)
        targets = [
                {
                    'boxes': d['boxes'].to(device),
                    'labels': d['labels'].to(device)
                } 
            for d in data[1]]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        metric_collector1.append(loss_dict['loss_box_reg'].item())
        metric_collector2.append(loss_dict['loss_rpn_box_reg'].item())
        metric_collector3.append(loss_dict['loss_objectness'].item())
        metric_collector4.append(loss_dict['loss_classifier'].item())
        st = time.time()
        # https://pytorch.org/vision/stable/models/faster_rcnn.html
        #   
        # See: https://github.com/pytorch/vision/blob/main/torchvision/models/detection/faster_rcnn.py
        #   class FasterRCNN

        # TODO: track componentwise (4) RCNN loss vs. epoch.

    # TODO: checkpoint model weights, optimizer state, and other as needed to resume
    # TODO: plot each of the 4 RCNN loss components vs epoch-number
        if idx % 5 == 0:
            print('Epoch: {} | Iteration: {} | Loss: {} | time elapsed: {}'.format(epoch, idx, losses.item(), time.time() - st))
            if idx % 20 == 0:
                plt.clf()
                plt.plot(metric_collector1) 
                plt.savefig('loss_box_reg.png')

                plt.clf()
                plt.plot(metric_collector2) 
                plt.savefig('loss_rpn_box_reg.png')

                plt.clf()
                plt.plot(metric_collector3) 
                plt.savefig('loss_objectness.png')

                plt.clf()
                plt.plot(metric_collector4) 
                plt.savefig('loss_classifier.png')

                if idx % 100 == 0:
                    os.makedirs('checkpoints', exist_ok=True)
                    torch.save(model.state_dict(), 'checkpoints/fasterRCNN_latest.pth')
                    evaluate(model, dataloader, device, idx)

    # VALIDATE
    model.eval()
    evaluate(model, dataloader, device, epoch)

Epoch: 0 | Iteration: 0 | Loss: 1.3525304794311523 | time elapsed: 3.6716461181640625e-05
Epoch: 0 | Iteration: 5 | Loss: 1.081961750984192 | time elapsed: 1.9073486328125e-05
Epoch: 0 | Iteration: 10 | Loss: 1.5094009637832642 | time elapsed: 1.430511474609375e-05
Epoch: 0 | Iteration: 15 | Loss: 1.2710129022598267 | time elapsed: 1.6927719116210938e-05
Epoch: 0 | Iteration: 20 | Loss: 1.309859037399292 | time elapsed: 1.3589859008789062e-05
Epoch: 0 | Iteration: 25 | Loss: 1.5488532781600952 | time elapsed: 1.5497207641601562e-05
Epoch: 0 | Iteration: 30 | Loss: 1.41199791431427 | time elapsed: 1.7642974853515625e-05
Epoch: 0 | Iteration: 35 | Loss: 1.2931313514709473 | time elapsed: 1.811981201171875e-05
Epoch: 0 | Iteration: 40 | Loss: 1.935907006263733 | time elapsed: 2.09808349609375e-05
Epoch: 0 | Iteration: 45 | Loss: 1.6971559524536133 | time elapsed: 1.4066696166992188e-05
Epoch: 0 | Iteration: 50 | Loss: 1.731423258781433 | time elapsed: 1.4543533325195312e-05
Epoch: 0 | Ite